Modified from this excellent notebook: 
https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min

In [ ]:
import os

import cudf
import cupy as cp
import janestreet
import xgboost as xgb

In [ ]:
# Load and preprocess CSV with cuDF
train = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train.query('weight > 0')
train['action'] = (train['resp'] > 0.002).astype('int')
features = [c for c in train.columns if 'feature' in c]
X_train = train.loc[:, features]
y_train = train.loc[:, 'action']

In [ ]:
# Prepare data and train xgboost on GPU
params = dict(
    objective='binary:logistic',
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=42,
    tree_method='gpu_hist')
dtrain = xgb.DMatrix(X_train, y_train)
clf = xgb.train(params, dtrain, num_boost_round=500)

In [ ]:
# Initialize the environment and submit
env = janestreet.make_env()
for (test_df, pred_df) in env.iter_test():
    dtest = xgb.DMatrix(test_df.loc[:, features])
    pred_df.action = clf.predict(dtest).round().astype(int)
    env.predict(pred_df)